# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [ ]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a loop to get a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    

#### Processing the files to create the csv data file that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
# extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv \
# that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Apache Cassandra tables. 

## The <font color=red>event_datafile_new.csv</font> within the Workspace directory contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [ ]:
# This should make a connection to a Cassandra instance in your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster

try:
    cluster = Cluster()
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [ ]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [ ]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### Now we need to create tables to run three queries:

### Task 1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338 and itemInSession = 4.

#### For this query, we partition by sessionId and itemInSession, thus these columns becoming our partition key and composite primary key.

In [ ]:
query = "CREATE TABLE IF NOT EXISTS session_library "
query = query + """(sessionId int, itemInSession int, artist text, song text, length float,
                    PRIMARY KEY ((sessionId, itemInSession)))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
## Create table 1

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO session_library (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)

In [ ]:
## Verify that the data have been inserted into table 1 using SELECT

query = """SELECT artist, song, length FROM session_library 
           WHERE sessionId = 338 AND itemInSession = 4"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

### Task 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182.

#### For this query, we partion by userId and sessionId, thus these columns becoming our partition key. Furthermore, we sort by itemInSession, this becoming our clustering column. These three columns become our composite primary key.

In [ ]:
query = "CREATE TABLE IF NOT EXISTS user_library "
query = query + """(userId int, sessionId int, itemInSession int, 
                    firstName text, lastName text, artist text, song text, 
                    PRIMARY KEY ((userId, sessionId), itemInSession))"""
try:
    session.execute(query)
except Exception as e:
    print(e)              

In [ ]:
## Create table 2

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = """INSERT INTO user_library (userId, sessionId, itemInSession, 
                                             firstName, lastName, artist, song)"""
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), \
                                    line[1], line[4], line[0], line[9]))
        except Exception as e:
            print(e)

In [ ]:
## Verify that the data have been inserted into table 2 using SELECT

query = """SELECT artist, song, firstName, lastName FROM user_library 
           WHERE userId = 10 AND sessionId = 182"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

### Task 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'.

#### For this query, we partition by song, thus this column becoming our partition key. Furthermore, to make the key unique, we add userId as a clustering column. These two columns become our composite primary key.

In [ ]:
query = "CREATE TABLE IF NOT EXISTS listen_library "
query = query + """(song text, userId int, firstName text, lastName text, 
                    PRIMARY KEY (song, userId))"""
try:
    session.execute(query)
except Exception as e:
    print(e)                  

In [ ]:
## Create table 3

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO listen_library (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        try:
            session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        except Exception as e:
            print(e)

In [ ]:
## Verify that the data have been inserted into table 3 using SELECT

query = """SELECT firstName, lastName FROM listen_library 
           WHERE song = 'All Hands Against His Own'"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

### Drop the tables before closing out the sessions

In [ ]:
try: 
    cur.execute("DROP table session_library")
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)
try: 
    cur.execute("DROP table user_library")
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)
try: 
    cur.execute("DROP table listen_library")
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()